In [11]:
import glob
import nbformat
from nbformat.v4 import new_code_cell, new_markdown_cell, new_raw_cell
from nbconvert import HTMLExporter
from nbconvert.preprocessors import ExecutePreprocessor, CellExecutionError
import re
import textwrap
import shutil
import json
import pandas as pd
from pathlib import Path
from datetime import datetime


In [54]:
fpaths = glob.glob('../../accy575-sp-2023/02-pcard-submissions/*.ipynb')
fpaths = list(filter(lambda f: not f.endswith('-graded.ipynb'), fpaths))
fpaths[:5]

['../../accy575-sp-2023/02-pcard-submissions\\anacondas_324892_7729312_PCard-after-milestone-1.ipynb',
 '../../accy575-sp-2023/02-pcard-submissions\\ancillaries_334459_7718488_PCard_Ancillaries.ipynb',
 '../../accy575-sp-2023/02-pcard-submissions\\df_324476_7727330_PCard-after-milestone.ipynb',
 '../../accy575-sp-2023/02-pcard-submissions\\excellent_65151_7725650_PCard_Excellent.ipynb',
 '../../accy575-sp-2023/02-pcard-submissions\\get_an_a_323576_7703657_PCard-get_an_A.ipynb']

In [51]:
fpaths = glob.glob('../submissions/exercise-05/*.ipynb')
fpaths = list(filter(lambda f: not f.endswith('-graded.ipynb'), fpaths))
fpaths[:5]

['../submissions/exercise-05\\chenchloe_83497_7409117_exercise_05_pandas_filtering_sorting.ipynb',
 '../submissions/exercise-05\\esparzajoel_51499_7451315_Copy_of_exercise_05_pandas_filtering_sorting.ipynb',
 '../submissions/exercise-05\\tabornatalie_8175_7404582_exercise_05_pandas_filtering_sorting.ipynb']

In [55]:
from lambdagrader import *

graded_results = []

for notebook_path in fpaths:
    try:
        print('=============================')
        nb = nbformat.read(notebook_path, as_version=4)
        
        for cell in nb.cells:
            test_case_metadata = extract_test_case_metadata_from_cell(cell.source)

            if test_case_metadata:
                cell.source = convert_to_grader_code(cell.source)

        add_scripts_to_notebook(nb)
        
        print(f'Grading {notebook_path}')
        
        ep = ExecutePreprocessor(
            timeout=1800,
            kernel_name='python3',
            allow_errors=True
        )
        ep.preprocess(nb)
        
        # save graded notebook
        converted_notebook_path = notebook_path.replace('.ipynb', '-graded.ipynb')
        with open(converted_notebook_path, mode='w', encoding='utf-8') as f:
            nbformat.write(nb, f)
        
        # rename graded result JSON file
        graded_result_json_path = notebook_path.replace('.ipynb', '-result.json')
        shutil.move('lambdagrader-result.json', graded_result_json_path)
        
        # read graded result to generate a summary
        with open(graded_result_json_path, mode='r') as f:
            graded_result = json.load(f)
            
        graded_notebook_filename = Path(converted_notebook_path).name
            
        summary = ''
        summary += f"File: {graded_notebook_filename}\n"
        summary += f"Score: {graded_result['learner_score']} out of {graded_result['total_available']}\n"
        summary += f"Passed {graded_result['num_passed_cases']} out of {graded_result['num_test_cases']} test cases\n"
        summary += f"Grading took {graded_result['grading_duration_in_seconds']} seconds\n\n"
        summary += 'Test Case Summary\n'
        
        for o in graded_result['results']:
            summary += "-----------------\n"
            summary += f"{o['test_case_name']} {'passed' if o['pass'] else 'failed'}: {o['points']} out of {o['available_points']} points\n"
            
            if not o['pass']:
                summary += f"[Autograder Output]\n{o['message']}\n\n"
                
        result_summary = {
            'filename': graded_notebook_filename,
            'grading_finished_at': graded_result['grading_finished_at'],
            'grading_duration_in_seconds': graded_result['grading_duration_in_seconds'],
            'learner_score': graded_result['learner_score'],
            'total_available': graded_result['total_available'],
            'num_test_cases': graded_result['num_test_cases'],
            'num_passed_cases': graded_result['num_passed_cases'],
            'num_failed_cases': graded_result['num_failed_cases'],
            'summary': summary
        }

        # remove prepend, append cells added by LambdaGrader before storing to HTML
        nb.cells.pop(0)  # first cell (added by LambdaGrader)
        nb.cells.pop()   # last cell (added by LambdaGrader)
        
        insert_index = 0
        
        # add result summary
        nb.cells.insert(insert_index, new_markdown_cell('# 🧭 LambdaGrader Summary'))
        insert_index += 1
        
        nb.cells.insert(insert_index, new_markdown_cell('## Metadata'))
        insert_index += 1
        
        df_metadata = pd.DataFrame({
            'name': [
                'graded_filename',
                'grading_finished_at',
                'grading_duration',
                '**learner_score**',
                'max_score',
                'learner_score_in_percentage',
                'num_test_cases',
                'num_passed_cases',
                'num_failed_cases'
            ],
            'value': [
                graded_notebook_filename,
                graded_result['grading_finished_at'],
                f"{graded_result['grading_duration_in_seconds']} second{'' if graded_result['grading_duration_in_seconds'] == 0 else 's'}",
                f"**{graded_result['learner_score']}**",
                graded_result['total_available'],
                f"{round(graded_result['learner_score'] / graded_result['total_available'] * 100, 2)}%",
                graded_result['num_test_cases'],
                graded_result['num_passed_cases'],
                graded_result['num_failed_cases']
            ]
        })
        nb.cells.insert(insert_index, new_markdown_cell(df_metadata.to_markdown(index=False)))
        insert_index += 1
        
        nb.cells.insert(insert_index, new_markdown_cell('## Test case results'))
        insert_index += 1
        
        df_r = pd.DataFrame(graded_result['results'])
        df_r['pass'] = df_r['pass'].map({
            True: '✔️ Pass', False: '❌ Fail'
        })
        # 
        df_r.rename(columns={
            'available_points': 'max_score',
            'points': 'learner_score',
            'pass': 'result'
        }, inplace=True)
        
        nb.cells.insert(insert_index, new_markdown_cell(df_r.to_markdown()))
        insert_index += 1
        
        nb.cells.insert(insert_index, new_markdown_cell('\n---\n'))
        insert_index += 1
        
        # store graded result to HTML
        graded_html_path = notebook_path.replace('.ipynb', '-graded.html')
        html_exporter = HTMLExporter()
        r = html_exporter.from_notebook_node(nb)
        with open(graded_html_path, 'w', encoding="utf-8") as f:
            f.write(r[0])
        
        graded_results.append(result_summary)
        
        print(f'Complete')
    except CellExecutionError as e:
        print(f'CellExecutionError on {notebook_path}')
        print('-----------------------------')
        print(e)
        
df_summary = pd.DataFrame(graded_results)

df_summary.to_csv(
    f"graded_result_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv",
    index=None
)

Grading ../../accy575-sp-2023/02-pcard-submissions\anacondas_324892_7729312_PCard-after-milestone-1.ipynb
Complete
Grading ../../accy575-sp-2023/02-pcard-submissions\ancillaries_334459_7718488_PCard_Ancillaries.ipynb
Complete
Grading ../../accy575-sp-2023/02-pcard-submissions\df_324476_7727330_PCard-after-milestone.ipynb
Complete
Grading ../../accy575-sp-2023/02-pcard-submissions\excellent_65151_7725650_PCard_Excellent.ipynb
Complete
Grading ../../accy575-sp-2023/02-pcard-submissions\get_an_a_323576_7703657_PCard-get_an_A.ipynb
Complete
Grading ../../accy575-sp-2023/02-pcard-submissions\group_x_6126_7702498_PCard-after-milestone_Group_X .ipynb
Complete
Grading ../../accy575-sp-2023/02-pcard-submissions\may_323193_7713466_PCard7.ipynb
Complete
Grading ../../accy575-sp-2023/02-pcard-submissions\numpy_321946_7726318_Group Numpy_Case 2 P-Card Python Portion.ipynb
Complete
Grading ../../accy575-sp-2023/02-pcard-submissions\pandas_330521_7726659_Pandas_PCard.ipynb
Complete
Grading ../../accy

In [ ]:
from lambdagrader import *

tcs = extract_test_cases_metadata_from_notebook( '../notebooks\\case-study-04-rideshare-trips-SOLUTION.ipynb')

s = 0

for o in tcs:
    s += o['points']
    
print(s)